In [1]:
import torch

In [46]:
from model.othermodels import CK_LgNet

In [47]:
model = CK_LgNet()

this is pre-set model, we disable all config


In [51]:
[name for name, p in model.named_parameters() if 'beta' in name ]

[]

In [44]:
optimizer = Tiger([{'params': [p for name, p in model.named_parameters() if 'bias' in name ],'type':'tensor_adding'},
                   {'params': [p for name, p in model.named_parameters() if 'bias' not in name ],'type':'tensor_contraction'}\
                  ],lr =0.001)

In [37]:
a = torch.randn(10,2)
b = torch.randn(10,2)

In [45]:
loss = torch.nn.MSELoss()(model(a),b)
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [29]:
torch.randn(1,2,3,4,5).norm(dim=-1,keepdim=True).shape

torch.Size([1, 2, 3, 4, 1])

In [42]:
from typing import Tuple, Optional, Callable

import torch
from torch.optim.optimizer import Optimizer

# functions

def exists(val):
    return val is not None

# update functions

def update_fn(p, grad, m, lr, wd, beta1, beta2):
    ### do dynamic lr configuration as the name of parameter
    #### 'bias|beta|gamma' -> 0.5*lr
    #### 'embeddings' -> lr*sqrt(p.)
    #### 'others'     -> lr*sqrt(p.)
    p.data.mul_(1 - lr * wd)
    m.mul_(beta1).add(grad, alpha = 1 - beta2)
    p.add(m.sign_(),alpha=-lr)

# class
 
class Tiger(Optimizer):
    def __init__(self,params,
        lr: float = 1e-4,
        betas: Tuple[float, float] = (0.95, 0.95),
        weight_decay: float = 0.0
    ):
        assert lr > 0.
        assert all([0. <= beta <= 1. for beta in betas])
        assert len(params) == 2, print('please provide 2 type parameter: tensor_adding| tensor_contraction. By {\'type\':\'tensor_adding\'}')
        assert betas[0] == betas[1], print("tiger optimizer requires same beta1 and beta2")
        
        defaults = dict(
            lr = lr,
            betas = betas,
            weight_decay = weight_decay
        )
        
        super().__init__(params, defaults)

        self.update_fn = update_fn

    @torch.no_grad()
    def step(
        self,
        closure: Optional[Callable] = None
    ):

        loss = None
        if exists(closure):
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            
            for p in filter(lambda p: exists(p.grad), group['params']):

                grad, lr, wd, beta1, beta2, state = p.grad, group['lr'], group['weight_decay'], *group['betas'], self.state[p]
                if group['type'] =='bias':
                    lr, wd = lr*0.5 , 0
                else:
                    lr = lr*p.data.norm()
                # init state - exponential moving average of gradient values

                if len(state) == 0:
                    state['exp_avg'] = torch.zeros_like(p)

                exp_avg = state['exp_avg']

                self.update_fn(
                    p,
                    grad,
                    exp_avg,
                    lr,
                    wd,
                    beta1,
                    beta2
                )

        return loss